In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import sys
sys.path.append("./base_model.py")
from base_model import BaseModel

In [9]:
import torchvision
import torch.optim as optim
import torch
import torch.nn as nn
import torchvision.transforms as transforms

In [10]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU avalable")
    torch.cuda.set_device(0)  # You can specify the GPU device index if you have multiple GPUs
else:
    device = torch.device("cpu")


GPU avalable


In [11]:
transformer =transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the images to (224, 224)
    transforms.ToTensor()])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transformer )
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True,transform=transformer)
testloader = torch.utils.data.DataLoader(testset, batch_size=128,
                                         shuffle=False, num_workers=2)





Using downloaded and verified file: ./data/cifar-10-python.tar.gz
Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [12]:
AlexNet = BaseModel()

additional_classifier = nn.Linear(AlexNet.num_features, 10) 
AlexNet.add_classifier(7,additional_classifier)
main_modal = AlexNet.model
main_modal.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(main_modal.parameters(), lr=0.001, momentum=0.9)

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:02<00:00, 101MB/s]


In [13]:
main_modal.train()
for epoch in range(10):  # Train for 10 epochs
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = main_modal(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print('Epoch [%d], Loss: %.4f' % (epoch + 1, running_loss / (i + 1)))
torch.save(main_modal, "./fine_tune.pt")

Epoch [1], Loss: 0.9597
Epoch [2], Loss: 0.5776
Epoch [3], Loss: 0.4794
Epoch [4], Loss: 0.4230
Epoch [5], Loss: 0.3789
Epoch [6], Loss: 0.3490
Epoch [7], Loss: 0.3247
Epoch [8], Loss: 0.2971
Epoch [9], Loss: 0.2733
Epoch [10], Loss: 0.2573


In [16]:
main_modal.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = main_modal(images)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

accuracy_finetune = 100 * correct / total
print('Test accuracy after fine-tuning: {:.2f}%'.format(accuracy_finetune))


Test accuracy after fine-tuning: 89.30%
